In [7]:
import rope.base.project
from rope.refactor.extract import ExtractMethod
import re

APOSTROPHE_MARKER = "__APOSTROPHE__"
PERCENT_FORMAT_MARKER = "__PERCENT_FORMAT__"

def serialize(text: str):
    # Replace "'{var}'" with "__APOSTROPHE__{var}__APOSTROPHE__"
    text = re.sub(r"'{([^'}]*?)}'", f"{APOSTROPHE_MARKER}{{\\1}}{APOSTROPHE_MARKER}", text)
    # Replace "%s" with "__PERCENT_FORMAT__"
    text = re.sub(r"%\((.*?)\)s", f"{PERCENT_FORMAT_MARKER}{{\\1}}", text)
    return text

def deserialize(text: str):
    text = re.sub(f"{APOSTROPHE_MARKER}{{(.*?)}}{APOSTROPHE_MARKER}", "'{\\1}'", text)
    text = re.sub(f"{PERCENT_FORMAT_MARKER}{{(.*?)}}", "%(\\1)s", text)
    return text

myproject = rope.base.project.Project('../../')

myresource = myproject.get_resource('tests/notebooks/src/test.py')
contents = myresource.read()
serialized_contents = serialize(myresource.read())
myresource.write(serialized_contents)
extract_span = r"""if BASERUN_API_KEY is not None:
    pass

OPENAI_TIMEOUT = 60  # one minute

OPENAI_EXCLUSIVE_MODELS = [
    "gpt-4-1106-preview",
    "gpt-3.5-turbo-1106",
    "gpt-3.5-turbo-1106",
]
SEED = 100


class OpenAIProxy:
    def __init__(self):
        pass"""
serialized_extract_span = serialize(extract_span)
print(serialized_extract_span)

start, end = serialized_contents.find(serialized_extract_span), serialized_contents.find(serialized_extract_span) + len(serialized_extract_span)
print(start, end)

try:
    extractor = ExtractMethod(myproject, myresource, start, end)
    change_set = extractor.get_changes("helper", similar=True)
    for change in change_set.changes:
        if change.old_contents is not None:
            change.old_contents = deserialize(change.old_contents)
        else:
            change.old_contents = deserialize(change.resource.read())
        change.new_contents = deserialize(change.new_contents)
    for change in change_set.changes:
        print(change.get_description())
except Exception as e:
    print(e)
finally:
    myresource.write(contents)

if BASERUN_API_KEY is not None:
    pass

OPENAI_TIMEOUT = 60  # one minute

OPENAI_EXCLUSIVE_MODELS = [
    "gpt-4-1106-preview",
    "gpt-3.5-turbo-1106",
    "gpt-3.5-turbo-1106",
]
SEED = 100


class OpenAIProxy:
    def __init__(self):
        pass
377 630
--- a/tests/notebooks/src/test.py
+++ b/tests/notebooks/src/test.py
@@ -16,6 +16,25 @@
     OPENAI_API_VERSION,
 )
 from sweepai.logn.cache import file_cache
+
+
+def helper():
+    if BASERUN_API_KEY is not None:
+        pass
+
+    OPENAI_TIMEOUT = 60  # one minute
+
+    OPENAI_EXCLUSIVE_MODELS = [
+        "gpt-4-1106-preview",
+        "gpt-3.5-turbo-1106",
+        "gpt-3.5-turbo-1106",
+    ]
+    SEED = 100
+
+
+    class OpenAIProxy:
+        def __init__(self):
+            pass
 
 if BASERUN_API_KEY is not None:
     pass



In [22]:
len(change.new_contents) - len(change.old_contents)

85